In [1]:
import numpy as np
import torch

In [2]:
class Cavity_2D_dataset_for_GNOT():
    def __init__(self, data_path, L=1.0, sub_x = 1, train=True, normalize_y=False, y_normalizer=None, normalize_x = False, x_normalizer = None, up_normalizer =None, vertex = False, boundaries = False):

        '''
        This class takes in a dataset structure used by FNO and PINO models. 
        The dataset is typically stored as [Batch, Timesteps, X-coords, Y-coords]
        This function creates a Mesh Grid of coordinates (which is the same for all batches)
        and reshapes it into a linear with shape [Batch, number of nodes, dims(coordinates)].
        NOTE This array will then be the Transformer Queries Array (X or g).

        Similarly, the ground truth voriticity profiles are taken and split by ic_t_steps into
        both the output shape/ training validation loss dataset (Y) and the initial conditions 
        which will be the first ic_t_steps time steps in the dataset.
        NOTE This ic array will be nested in a tuple and called as an input function
        i.e. The Transformer Keys and Values Array (f or g_u).

        Finally, Theta is set a as a small zero array as the current model handling does not take
        a null input. This should not impact the model training (although it does make the datasets
        slighly larger which is not ideal for memory)
        '''
        # Normalizer settings:
        self.normalize_y = normalize_y
        self.normalize_x = normalize_x
        self.y_normalizer = y_normalizer
        self.x_normalizer = x_normalizer
        self.up_normalizer = up_normalizer
        self.sub_x = sub_x
        self.vertex = vertex
        self.boundaries = boundaries

        # Load in Dataset and retrieve shape
        self.data_out   = np.load(data_path)
        if self.sub_x > 1: self.subsampler()
        if self.vertex: self.cell_to_vertex_converter()
        if self.boundaries: self.add_boundaries()

        print(f'Dataset Shape: {self.data_out.shape}, subsampled by {self.sub_x}')
        # NOTE this can also be in the form of reynolds number 
        self.data_lid_v = np.round(np.arange(0.5,100.5,0.5),1) * 0.1/0.01 #<- apply for Reynolds Number
        self.n_batches  = self.data_out.shape[0]
        self.nx         = int(self.data_out.shape[1])
        self.num_nodes  = self.nx**2

        self.L = L
        self.train = train

    def assign_data_split_type(self, inference=True, train_ratio=0.7, seed=42):
        self.seed = seed
        self.data_split = train_ratio
        self.inference = inference

    def process(self):
        
        # SECTION 0: Split into train or test (Same as for FNO training)
        train_size = int(self.data_split * self.n_batches)
        test_size = self.n_batches - train_size

        seed_generator = torch.Generator().manual_seed(self.seed)

        # Perform Inference or Extrapolation (Inference is randomly sampled)
        if self.inference:
            train_dataset,  test_dataset    = torch.utils.data.random_split(torch.from_numpy(self.data_out),    [train_size, test_size], generator=seed_generator)
            train_lid_v,    test_lid_v      = torch.utils.data.random_split(torch.from_numpy(self.data_lid_v),  [train_size, test_size], generator=seed_generator)
            
            # The torch.utils.data.random_split() only gives objects with the whole datset or a integers, so we need to override these variables with the indexed datset split
            train_dataset,  test_dataset    = train_dataset.dataset[train_dataset.indices,...], test_dataset.dataset[test_dataset.indices,...]
            train_lid_v,    test_lid_v      = train_lid_v.dataset[train_lid_v.indices], test_lid_v.dataset[test_lid_v.indices]
            print(f'''Dataset Split up using torch generator seed: {seed_generator.initial_seed()}
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed({seed_generator.initial_seed()})\n ''')
        else:
            train_dataset,  test_dataset    = torch.from_numpy(self.data_out[:train_size,...]), torch.from_numpy(self.data_out[train_size:,...])
            train_lid_v,    test_lid_v      = torch.from_numpy(self.data_out[:test_size,...]), torch.from_numpy(self.data_out[test_size:,...])

        

        if self.train:
            self.data_out   = train_dataset
            self.data_lid_v = train_lid_v
            self.n_batches  = train_size
        else:
            self.data_out   = test_dataset
            self.data_lid_v = test_lid_v
            self.n_batches  = test_size

        # SECTION 1: Transformer Queries
        # Assume Isotropic Grid adjusting coordinates for cell centered or vertex points accordingly.
        # Also includes boundaries if stated (note boundaries + cell-centered will cause boundary coordinates to be 0-dx, 1+dx overflow)
        # this is to maintain isotropic property
        divisor = self.nx - 2*int(self.boundaries) + 1*int(self.vertex)
        dx = self.L/divisor
        offset = dx/2 - dx*int(self.boundaries) + dx/2*int(self.vertex)
        x = torch.arange(self.nx)/divisor + offset
        y = x

        # take note of the indexing. Best for this to match the output
        [X, Y] = torch.meshgrid(x, y, indexing = 'ij')
        print(X.shape)
        X = X.reshape(self.num_nodes,1)
        Y = Y.reshape(self.num_nodes,1)

        # we need to linearize these matrices.
        self.X_for_queries = torch.concat([Y,X],dim=-1)
        print('Queries', self.X_for_queries.shape, 'Coordinates', X.shape)
        
        # SECTION 3: Transform to be MIOdataset Loader Compatible
        self.normalizer()
        self.__update_dataset_config()

    def subsampler(self):
        self.data_out = torch.nn.functional.avg_pool2d(torch.tensor(self.data_out).permute(0,3,1,2), self.sub_x).permute(0,2,3,1).numpy()

    def cell_to_vertex_converter(self):
        self.data_out = torch.nn.functional.avg_pool2d(torch.tensor(self.data_out).permute(0,3,1,2),2,stride=1).permute(0,2,3,1).numpy()
    
    def add_boundaries(self):
        self.data_out = torch.nn.functional.pad(torch.tensor(self.data_out).permute(0,3,1,2),(1, 1, 1, 1)).permute(0,2,3,1).numpy()

        # Lid Velocity
        self.data_out[:,-1 ,:,0] = 1

        # Pressure
        self.data_out[:,  0 ,1:-1, 2] = self.data_out[:,  1 ,1:-1, 2]  # Bottom Wall
        self.data_out[:, -1 ,1:-1, 2] = self.data_out[:, -2 ,1:-1, 2]  # Lid (y-vel)
        self.data_out[:,1:-1,  0 , 2] = self.data_out[:,1:-1,  1 , 2]  # Left Wall
        self.data_out[:,1:-1, -1 , 2] = self.data_out[:,1:-1, -2 , 2]  # Right Wall

    def normalizer(self):
        if self.normalize_y:
            self.__normalize_y()
        if self.normalize_x:
            self.__normalize_x()

        self.__update_dataset_config()
        

    def __normalize_y(self):
        if self.y_normalizer is None:
            if self.normalize_y == 'unit':
                self.y_normalizer = UnitTransformer(self.data_out)
                print('Target features are normalized using unit transformer')
            else: 
                raise NotImplementedError
        else:
            self.data_out = self.y_normalizer.transform(self.data_out, inverse=False)  # a torch quantile transformer
            print('Target features are normalized using unit transformer')

    def __normalize_x(self):
        if self.x_normalizer is None:
            if self.normalize_x == 'unit':
                self.x_normalizer = UnitTransformer(self.X_for_queries)
                self.up_normalizer = UnitTransformer(self.data_lid_v)
            else: 
                raise NotImplementedError

    def __update_dataset_config(self):

        self.config = {
            'input_dim': self.X_for_queries.shape[-1],
            #'theta_dim': self.data_lid_v.shape[1],
            'output_dim': self.data_out.shape[-1]#,
            #'branch_sizes': [x.shape[1] for x in self.inputs_f[0]] if isinstance(self.inputs_f, list) else 0
        }

class UnitTransformer():
    def __init__(self, X):
        self.mean = X.mean(dim=0, keepdim=True)
        self.std = X.std(dim=0, keepdim=True) + 1e-8


    def to(self, device):
        self.mean = self.mean.to(device)
        self.std = self.std.to(device)
        return self

    def transform(self, X, inverse=True,component='all'):
        if component == 'all' or 'all-reduce':
            if inverse:
                orig_shape = X.shape
                return (X*(self.std - 1e-8) + self.mean).view(orig_shape)
            else:
                return (X-self.mean)/self.std
        else:
            if inverse:
                orig_shape = X.shape
                return (X*(self.std[:,component] - 1e-8)+ self.mean[:,component]).view(orig_shape)
            else:
                return (X - self.mean[:,component])/self.std[:,component]

In [3]:
data = Cavity_2D_dataset_for_GNOT(data_path= r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\GNOT\data\steady_cavity_case_b200_maxU100ms_simple_normalized.npy', 
                                    L=1.0, 
                                    sub_x = 8, 
                                    train=True, 
                                    normalize_y='unit', 
                                    y_normalizer=None, 
                                    normalize_x = 'unit', 
                                    x_normalizer = None, 
                                    up_normalizer =None, 
                                    vertex = True, 
                                    boundaries = True)


Dataset Shape: (200, 33, 33, 3), subsampled by 8


In [4]:
data.assign_data_split_type(inference=True, train_ratio=0.7, seed=42)
data.process()

Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([33, 33])
Queries torch.Size([1089, 2]) Coordinates torch.Size([1089, 1])
Target features are normalized using unit transformer


In [5]:
data.data_out.shape

torch.Size([140, 33, 33, 3])

# Calibrate input/output with model

In [28]:
from models.noahs_model import CGPTNO

# 2. Construct Model
model_args = dict()
model_args['trunk_size']        = data.config['input_dim']
#model_args['theta_size']        = data.config['theta_dim']
model_args['branch_sizes']      = [1]

model_args['output_size']         = 3
model_args['n_layers']            = 3
model_args['n_hidden']            = 128  
model_args['n_head']              = 1
model_args['attn_type']           = 'linear'
model_args['ffn_dropout']         = 0.0
model_args['attn_dropout']        = 0.0
model_args['mlp_layers']          = 2
model_args['act']                 = 'gelu'
model_args['hfourier_dim']        = 0

model = None
model = CGPTNO(
            trunk_size          = model_args['trunk_size'],# + model_args['theta_size'],
            branch_sizes        = model_args['branch_sizes'],     # No input function means no branches
            output_size         = model_args['output_size'],
            n_layers            = model_args['n_layers'],
            n_hidden            = model_args['n_hidden'],
            n_head              = model_args['n_head'],
            attn_type           = model_args['attn_type'],
            ffn_dropout         = model_args['ffn_dropout'],
            attn_dropout        = model_args['attn_dropout'],
            mlp_layers          = model_args['mlp_layers'],
            act                 = model_args['act'],
            horiz_fourier_dim   = model_args['hfourier_dim']
            )#.to(device)

In [7]:
data.data_lid_v.shape

torch.Size([140])

In [8]:
data.data_lid_v.unsqueeze(-1).unsqueeze(-1).shape

torch.Size([140, 1, 1])

In [41]:
out = model(x=data.X_for_queries.unsqueeze(0),inputs = data.data_lid_v[0].unsqueeze(0).unsqueeze(0).unsqueeze(0).float())

x.shape: torch.Size([1, 1089, 128]), z.shape: torch.Size([1, 1, 128])
x.shape: torch.Size([1, 1089, 128]), z.shape: torch.Size([1, 1, 128])
x.shape: torch.Size([1, 1089, 128]), z.shape: torch.Size([1, 1, 128])


In [55]:
print(data.data_lid_v[0].unsqueeze(0).unsqueeze(0).unsqueeze(0).float().shape)
print(data.data_lid_v[0].reshape([1,1,1]).float().shape)

torch.Size([1, 1, 1])
torch.Size([1, 1, 1])


In [14]:
from data_storage.cavity_2d_data_handling import Cavity_2D_dataset_handling_v2

dataset_args = dict()
training_args = dict()

# 1. Prepare Data
dataset_args['file']                    = r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\GNOT\data\steady_cavity_case_b200_maxU100ms_simple_normalized.npy'
dataset_args['percent split (decimal)'] = 0.7
dataset_args['randomizer seed']         = 42
dataset_args['use-normalizer']          = 'unit'
dataset_args['normalize_x']             = 'unit'
#dataset_args['subsampler']              = 4
dataset_args['cell to pointwise']       = True
dataset_args['add boundaries']          = True

dataset = Cavity_2D_dataset_handling_v2(dataset_args['file'], name='cavity', train=True, sub_x = 4,
                                    normalize_y=dataset_args['use-normalizer'], normalize_x = dataset_args['normalize_x'],
                                    data_split = dataset_args['percent split (decimal)'], seed = dataset_args['randomizer seed'],
                                    vertex = dataset_args['cell to pointwise'], boundaries = dataset_args['add boundaries']
                                    )

Dataset Shape: (200, 65, 65, 3), subsampled by 4
Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([65, 65])
Queries torch.Size([4225, 2]) Coordinates torch.Size([4225, 1])
Target features are normalized using unit transformer

c:\Users\Noahc\AppData\Local\Programs\Python\Python38\lib\site-packages\dgl\heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(



tensor([[ 0.0093,  0.0004, -0.0328]]) tensor([[0.2326, 0.1816, 0.1040]])
Target features are normalized using unit transformer
Input features are normalized using unit transformer


In [29]:
from data_utils import MultipleTensors, MIODataLoader
training_dataloader = MIODataLoader(dataset, batch_size=1, shuffle=True, drop_last=False)

In [56]:
len(training_dataloader)

140

In [30]:
for batch_n, batch in enumerate(training_dataloader):
    a,b,c = batch
    break

In [39]:
print(a.ndata['x'].shape)
print(c[0].shape)

print('compared to')

print(data.X_for_queries.shape)
print(data.data_lid_v[0].unsqueeze(0).unsqueeze(0).unsqueeze(0).float().shape)

torch.Size([4225, 2])
torch.Size([1, 1, 1])
compared to
torch.Size([1089, 2])
torch.Size([1, 1, 1])


Now we have a match. The model works

ok nice. Lets write some functions:

In [ ]:
def default_model_args():

    model_args = dict()
    model_args['trunk_size']        = data.config['input_dim']
    #model_args['theta_size']        = data.config['theta_dim']
    model_args['branch_sizes']      = [1]

    model_args['output_size']         = 3
    model_args['n_layers']            = 3
    model_args['n_hidden']            = 128  
    model_args['n_head']              = 1
    model_args['attn_type']           = 'linear'
    model_args['ffn_dropout']         = 0.0
    model_args['attn_dropout']        = 0.0
    model_args['mlp_layers']          = 2
    model_args['act']                 = 'gelu'
    model_args['hfourier_dim']        = 0

    return model_args



In [52]:
range(dataset.data_out.shape[0])

range(0, 140)

In [54]:
dataset.data_lid_v.shape

torch.Size([140])

In [ ]:
loss_f = torch.nn.MSELoss()

Lets look at a trial dictionary output:

In [59]:
output = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\GNOT\checkpoints\gnot_artemis\test_results.npy',allow_pickle=True).item()

In [60]:
output

{'Model Configuration': {'trunk_size': 2,
  'branch_sizes': [1],
  'output_size': 3,
  'n_layers': 3,
  'n_hidden': 128,
  'n_head': 1,
  'attn_type': 'linear',
  'ffn_dropout': 0.0,
  'attn_dropout': 0.0,
  'mlp_layers': 2,
  'act': 'gelu',
  'hfourier_dim': 0},
 'Training Configuration': {'epochs': 1,
  'save_dir': 'gnot_artemis',
  'base_lr': 0.001,
  'weight-decay': 5e-05,
  'grad-clip': 1000.0,
  'save_name': 'test'},
 'Data Configuration': {'file': 'C:\\Users\\Noahc\\Documents\\USYD\\PHD\\8 - Github\\GNOT\\data\\steady_cavity_case_b200_maxU100ms_simple_normalized.npy',
  'percent split (decimal)': 0.7,
  'randomizer seed': 42,
  'Interpolate (instead of Extrapolate)': True,
  'use-normalizer': 'unit',
  'normalize_x': 'unit',
  'cell to pointwise': True,
  'add boundaries': True,
  'sub_x': 4},
 'Epoch Time': [0.5232278999999997],
 'Training L2 Loss': [0.0700981542468071]}